In [231]:
import os
import sys
from compmusic.dunya import conn
import os
import codecs
import compmusic
from compmusic import dunya as dn
from compmusic.dunya import hindustani as hi
from compmusic.dunya import carnatic as ca
from compmusic.dunya import docserver as ds
from compmusic import musicbrainz
import json
import numpy as np
import requests
import json
import pandas as pd

In [182]:
token = "" #<get your api token form: https://dunya.compmusic.upf.edu/user/profile/>

# Ids of CC collection for both Carnatic and Hindustani music tradition
carnatic_cc = dict(id='a163c8f2-b75f-4655-86be-1504ea2944c2',
                  name='carnatic')
hindustani_cc = dict(id='6adc54c6-6605-4e57-8230-b85f1de5be2b',
                  name='hindustani')
collections = [carnatic_cc, hindustani_cc]

In [192]:
headers = {"Authorization": "Token %s" % token}

In [184]:
def get_mbids_in_collection(collection_id, music_tradition, token):
    """
    fetches mbids in a collection
    """
    if music_tradition == 'hindustani':
        tradition = hi
    elif music_tradition == 'carnatic':
        tradition = ca        
    
    dn.set_token(token)
    tradition.set_collections([collection_id])
    recs = tradition.get_recordings()
    return [r['mbid'] for r in recs]

In [230]:
output = []
for collection in collections[:1]:
    mbids = get_mbids_in_collection(collection['id'], collection['name'], token)
    for mbid in mbids:
        try:
            path = "document/by-id/%s" % (mbid)        
            url = conn._make_url(path)
            out = requests.get(url, headers=headers)
            content = json.loads(out.content)
            mapp = dict(zip(content['sourcefiles'], np.ones(len(content['sourcefiles']))))
            mapp.update(dict(mbid=mbid, collection=collection['name']))
        except:
            print("Issue with: %s, %s"%(collection['id'], mbid))
        output.append(mapp)

Issue with: a163c8f2-b75f-4655-86be-1504ea2944c2, c79c3d8f-29a2-40cd-85f7-cc98da4b4532
Issue with: a163c8f2-b75f-4655-86be-1504ea2944c2, 14c18b73-88a7-4092-9317-13ff31254b35


In [232]:
output_df = pd.DataFrame(output)

In [237]:
carn = output_df[output_df.collection=='carnatic']

In [244]:
carn[['tempo-manual', 'sections-manual', 'pitch-vocal', 'sections-manual-p',
       'bpm-manual', 'mphrases-manual', 'sama-manual', 'mp3',
       'multitrack-mridangam-right', 'multitrack-vocal',
       'multitrack-mridangam-left', 'multitrack-ghatam', 'multitrack-violin',
       'zip', 'multitrack-vocal-s']].sum().astype(int)

tempo-manual                  133
sections-manual               119
pitch-vocal                    56
sections-manual-p             119
bpm-manual                    119
mphrases-manual               117
sama-manual                   141
mp3                           251
multitrack-mridangam-right    170
multitrack-vocal              170
multitrack-mridangam-left     170
multitrack-ghatam              46
multitrack-violin             170
zip                            84
multitrack-vocal-s             24
dtype: int64

In [242]:
audio 197
pitch 197
pitch_vocal 56
tonic 197
sama 141
bpm 119
tempo 133
sections 119
phrases 117

'audio 197\npitch 197\npitch_vocal 56\ntonic 197\nsama 141\nbpm 119\ntempo 133\nsections 119\nphrases 117'

In [ ]:
class Saraga